In [3]:
import sys
sys.path.insert(0, r"C:\Users\michellexu\Pulse\engine\src\python\pulse\rl-hemorrhage-resuscitation\gating")

from torch import nn
import os
import torch
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import joblib

In [4]:
script_dir = script_dir = os.getcwd()
parent_dir = os.path.dirname(script_dir)

data = pd.read_csv(os.path.join(script_dir, "train_data_bv.csv"))

# data["observations"] = (
#     data["observations"]
#     .str.replace("],", "", regex=False)
#     .str.replace("array", "", regex=False)
#     .str.replace("(", "", regex=False)
#     .str.replace(")", "", regex=False)
#     .str.replace(", dtype=float32", "", regex=False)
#     .str.replace("dtype=float32", "", regex=False)
#     .str.replace("[", "", regex=False)
# )
# data["observations"] = data["observations"].apply(lambda x: "[" + x + "]")
# data["observations"] = data["observations"].apply(lambda x: np.array(ast.literal_eval(x), dtype=float))



In [5]:
data

,bv1,bv2,bv3,severity,label
0,4672.020119,4661.731009,4651.516970,0.05,0
1,4672.020119,4649.813507,4627.959100,0.10,0
2,4672.020119,4636.956158,4602.775980,0.15,0
3,4672.020119,4652.312035,4633.303977,0.50,0
4,4672.020119,4646.909993,4622.921291,0.60,0
...,...,...,...,...,...
2095,3755.683265,3680.153955,3609.281895,0.30,1
2096,3755.683265,3744.707314,3734.006134,0.35,1
2097,3755.683265,3648.306231,3550.103007,0.40,1
2098,3755.683265,3718.991026,3684.734201,0.90,1


In [9]:
expanded = pd.DataFrame(data["observations"].tolist(), index=data.index)
expanded.columns = ["HR1", "MAP1", "SAP1", "OxSat1", "ETCO21", "RR1", "Temp1", "age1", "bmi1", "sex1", "HR2", "MAP2", "SAP2", "OxSat2", "ETCO22", "RR2", "Temp2", "age2", "bmi2", "sex2", "HR3", "MAP3", "SAP3", "OxSat3", "ETCO23", "RR3", "Temp3", "age3", "bmi3", "sex3"]
expanded

,HR1,MAP1,SAP1,OxSat1,ETCO21,RR1,Temp1,age1,bmi1,sex1,...,HR3,MAP3,SAP3,OxSat3,ETCO23,RR3,Temp3,age3,bmi3,sex3
0,71.0,83.105870,102.16185,0.975392,35.880207,12.048193,32.870560,60.0,21.0,0.0,...,72.486420,82.865090,101.188034,0.972136,33.766020,12.048193,32.865044,60.0,21.0,0.0
1,71.0,83.105870,102.16185,0.975392,35.880207,12.048193,32.870560,60.0,21.0,0.0,...,72.517070,82.837820,101.155620,0.972140,33.764423,12.048193,32.865036,60.0,21.0,0.0
2,71.0,83.105870,102.16185,0.975392,35.880207,12.048193,32.870560,60.0,21.0,0.0,...,72.550480,82.808200,101.120380,0.972144,33.762672,12.048193,32.865067,60.0,21.0,0.0
3,71.0,83.105870,102.16185,0.975392,35.880207,12.048193,32.870560,60.0,21.0,0.0,...,72.572840,82.839260,101.163490,0.972133,33.766846,12.048193,32.865060,60.0,21.0,0.0
4,71.0,83.105870,102.16185,0.975392,35.880207,12.048193,32.870560,60.0,21.0,0.0,...,72.604450,82.825860,101.149120,0.972133,33.766666,12.048193,32.865032,60.0,21.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,85.0,99.687195,118.93690,0.975460,35.878310,12.048193,32.777435,44.0,20.0,1.0,...,85.497665,96.414696,116.129196,0.970246,33.650837,12.048193,32.767303,44.0,20.0,1.0
2096,85.0,99.687195,118.93690,0.975460,35.878310,12.048193,32.777435,44.0,20.0,1.0,...,85.480230,96.857330,116.634990,0.970207,33.666847,12.048193,32.767010,44.0,20.0,1.0
2097,85.0,99.687195,118.93690,0.975460,35.878310,12.048193,32.777435,44.0,20.0,1.0,...,85.508400,96.175010,115.858920,0.970263,33.643417,12.048193,32.767350,44.0,20.0,1.0
2098,85.0,99.687195,118.93690,0.975460,35.878310,12.048193,32.777435,44.0,20.0,1.0,...,85.499100,96.612040,116.375050,0.970204,33.665207,12.048193,32.767160,44.0,20.0,1.0


In [10]:
#filtered = expanded[expanded["age1"] > 50]
#filtered = filtered[filtered["bmi1"]>25]
filtered = expanded
filtered["dMAP1"] = filtered["MAP1"] - filtered["MAP2"]
filtered["shock_index1"] = filtered["HR1"] / filtered["SAP1"]
filtered["shock_index2"] = filtered["HR2"] / filtered["SAP2"]
filtered

,HR1,MAP1,SAP1,OxSat1,ETCO21,RR1,Temp1,age1,bmi1,sex1,...,OxSat3,ETCO23,RR3,Temp3,age3,bmi3,sex3,dMAP1,shock_index1,shock_index2
0,71.0,83.105870,102.16185,0.975392,35.880207,12.048193,32.870560,60.0,21.0,0.0,...,0.972136,33.766020,12.048193,32.865044,60.0,21.0,0.0,0.084730,0.694976,0.706497
1,71.0,83.105870,102.16185,0.975392,35.880207,12.048193,32.870560,60.0,21.0,0.0,...,0.972140,33.764423,12.048193,32.865036,60.0,21.0,0.0,0.094586,0.694976,0.706587
2,71.0,83.105870,102.16185,0.975392,35.880207,12.048193,32.870560,60.0,21.0,0.0,...,0.972144,33.762672,12.048193,32.865067,60.0,21.0,0.0,0.105360,0.694976,0.706685
3,71.0,83.105870,102.16185,0.975392,35.880207,12.048193,32.870560,60.0,21.0,0.0,...,0.972133,33.766846,12.048193,32.865060,60.0,21.0,0.0,0.119820,0.694976,0.706848
4,71.0,83.105870,102.16185,0.975392,35.880207,12.048193,32.870560,60.0,21.0,0.0,...,0.972133,33.766666,12.048193,32.865032,60.0,21.0,0.0,0.132090,0.694976,0.706969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,85.0,99.687195,118.93690,0.975460,35.878310,12.048193,32.777435,44.0,20.0,1.0,...,0.970246,33.650837,12.048193,32.767303,44.0,20.0,1.0,2.834419,0.714665,0.728437
2096,85.0,99.687195,118.93690,0.975460,35.878310,12.048193,32.777435,44.0,20.0,1.0,...,0.970207,33.666847,12.048193,32.767010,44.0,20.0,1.0,2.754725,0.714665,0.727912
2097,85.0,99.687195,118.93690,0.975460,35.878310,12.048193,32.777435,44.0,20.0,1.0,...,0.970263,33.643417,12.048193,32.767350,44.0,20.0,1.0,2.889235,0.714665,0.728794
2098,85.0,99.687195,118.93690,0.975460,35.878310,12.048193,32.777435,44.0,20.0,1.0,...,0.970204,33.665207,12.048193,32.767160,44.0,20.0,1.0,2.858105,0.714665,0.728571


In [11]:
observations = np.stack(data["observations"].to_numpy())  # or np.vstack(list(data["observations"]))

# split data into train, val, test & shuffle
y_np = data.iloc[:, 2].to_numpy()
observations

array([[ 71.      ,  83.10587 , 102.16185 , ...,  60.      ,  21.      ,
          0.      ],
       [ 71.      ,  83.10587 , 102.16185 , ...,  60.      ,  21.      ,
          0.      ],
       [ 71.      ,  83.10587 , 102.16185 , ...,  60.      ,  21.      ,
          0.      ],
       ...,
       [ 85.      ,  99.687195, 118.9369  , ...,  44.      ,  20.      ,
          1.      ],
       [ 85.      ,  99.687195, 118.9369  , ...,  44.      ,  20.      ,
          1.      ],
       [ 85.      ,  99.687195, 118.9369  , ...,  44.      ,  20.      ,
          1.      ]], shape=(2100, 30))

In [103]:
X_np = np.concatenate((observations[:, :7], observations[:, 10:17], observations[:, 20:27]), axis=1)
X_np

array([[ 71.      ,  83.10587 , 102.16185 , ...,  33.76602 ,  12.048193,
         32.865044],
       [ 71.      ,  83.10587 , 102.16185 , ...,  33.764423,  12.048193,
         32.865036],
       [ 71.      ,  83.10587 , 102.16185 , ...,  33.762672,  12.048193,
         32.865067],
       ...,
       [ 85.      ,  99.687195, 118.9369  , ...,  33.643417,  12.048193,
         32.76735 ],
       [ 85.      ,  99.687195, 118.9369  , ...,  33.665207,  12.048193,
         32.76716 ],
       [ 85.      ,  99.687195, 118.9369  , ...,  33.664948,  12.048193,
         32.767113]], shape=(2100, 22))

In [99]:
shock_index_1 = X_np[:, 0] / (X_np[:, 2])
shock_index_2 = X_np[:, 7] / (X_np[:, 9])
X_np = np.insert(X_np, 0, shock_index_1, axis=1)
X_np = np.insert(X_np, 7, shock_index_2, axis=1)
X_np

array([[ 0.69497567, 71.        , 83.10587   , ..., 33.76602   ,
        12.048193  , 32.865044  ],
       [ 0.69497567, 71.        , 83.10587   , ..., 33.764423  ,
        12.048193  , 32.865036  ],
       [ 0.69497567, 71.        , 83.10587   , ..., 33.762672  ,
        12.048193  , 32.865067  ],
       ...,
       [ 0.71466467, 85.        , 99.687195  , ..., 33.643417  ,
        12.048193  , 32.76735   ],
       [ 0.71466467, 85.        , 99.687195  , ..., 33.665207  ,
        12.048193  , 32.76716   ],
       [ 0.71466467, 85.        , 99.687195  , ..., 33.664948  ,
        12.048193  , 32.767113  ]], shape=(2100, 23))

In [100]:
features = []
for i in range(15):
    delta = X_np[:, i] - X_np[:, i + 7]
    features.append(delta)
features = np.stack(features, axis=1)
X_np = features

In [101]:
X_np

array([[-1.15212117e-02,  3.81294400e+01,  1.14482900e+01, ...,
         1.61904000e-03,  4.66550000e-02, -9.60020000e-02],
       [-1.16110220e-02,  3.81294400e+01,  1.14460100e+01, ...,
         1.61585000e-03,  4.82140000e-02, -9.60020000e-02],
       [-1.17091574e-02,  3.81294400e+01,  1.14435200e+01, ...,
         1.61233000e-03,  4.99280000e-02, -9.60020000e-02],
       ...,
       [-1.41295992e-02,  5.22225650e+01,  1.47844850e+01, ...,
         1.96260000e-03,  1.30543000e-01, -4.81930000e-02],
       [-1.39058657e-02,  5.22225650e+01,  1.47840950e+01, ...,
         2.01445000e-03,  1.09341000e-01, -4.81930000e-02],
       [-1.40094244e-02,  5.22225650e+01,  1.47837950e+01, ...,
         2.01460000e-03,  1.09600000e-01, -4.81930000e-02]],
      shape=(2100, 15))

In [79]:
X_np = np.concatenate((X_np[:, :3], X_np[:, 4:5], X_np[:, 7:10], X_np[:, 11:12]), axis=1)

In [80]:
X_np

array([[-1.48642 ,  0.24078 ,  0.973816, ..., -0.116905, -0.100856,
         0.17579 ],
       [-1.51707 ,  0.26805 ,  1.00623 , ..., -0.087915, -0.04518 ,
         0.188033],
       [-1.55048 ,  0.29767 ,  1.04147 , ..., -0.05673 ,  0.01481 ,
         0.201595],
       ...,
       [-0.5084  ,  3.512185,  3.07798 , ..., -1.558884,  1.3234  ,
         0.36055 ],
       [-0.4991  ,  3.075155,  2.56185 , ..., -2.716665, -0.14904 ,
         0.228817],
       [-0.50197 ,  3.111985,  2.60096 , ..., -2.65623 , -0.07901 ,
         0.231838]], shape=(2100, 8))

In [81]:

X_train, X_temp, y_train, y_temp = train_test_split(X_np, y_np, test_size=0.3, random_state=42, shuffle=True, stratify=y_np)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True, stratify=y_temp)

X_train_np = X_train
X_val_np = X_val
X_test_np = X_test

# fit scaler on training data only
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_np)

# apply same transformation to val/test
X_val_scaled = scaler.transform(X_val_np)
X_test_scaled = scaler.transform(X_test_np)

# optionally save scaler for later use (inverse transform / inference)
joblib.dump(scaler, os.path.join(script_dir, "gating_scaler.pkl"))

# convert to torch tensors
X_train = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)


train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class GatingNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=8, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=32)
        self.fc4 = nn.Linear(in_features=32, out_features=2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.fc4(x)

model = GatingNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

n_epochs = 20
for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5 == 1:    # print every 2 mini-batches
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 5:.3f}")
            running_loss = 0.0

    # validation pass
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            #inputs = inputs.to(device)
            #labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_acc = correct / total if total > 0 else 0.0
    print(f"Finished epoch #{epoch + 1} val_loss: {avg_val_loss:.4f} val_acc: {val_acc:.4f}")

save_path = os.path.join(script_dir, "gating_model.pth")
torch.save(model.state_dict(), save_path)
print("Finished Training and saved model at", save_path)

[1, 2] loss: 0.278
[1, 7] loss: 0.693
[1, 12] loss: 0.694
Finished epoch #1 val_loss: 0.6903 val_acc: 0.5175
[2, 2] loss: 0.277
[2, 7] loss: 0.693
[2, 12] loss: 0.694
Finished epoch #2 val_loss: 0.6902 val_acc: 0.5175
[3, 2] loss: 0.278
[3, 7] loss: 0.693
[3, 12] loss: 0.693
Finished epoch #3 val_loss: 0.6900 val_acc: 0.5238
[4, 2] loss: 0.279
[4, 7] loss: 0.691
[4, 12] loss: 0.692
Finished epoch #4 val_loss: 0.6898 val_acc: 0.5206
[5, 2] loss: 0.278
[5, 7] loss: 0.693
[5, 12] loss: 0.691
Finished epoch #5 val_loss: 0.6897 val_acc: 0.5302
[6, 2] loss: 0.277
[6, 7] loss: 0.692
[6, 12] loss: 0.691
Finished epoch #6 val_loss: 0.6895 val_acc: 0.5333
[7, 2] loss: 0.278
[7, 7] loss: 0.690
[7, 12] loss: 0.693
Finished epoch #7 val_loss: 0.6894 val_acc: 0.5333
[8, 2] loss: 0.277
[8, 7] loss: 0.692
[8, 12] loss: 0.691
Finished epoch #8 val_loss: 0.6892 val_acc: 0.5270
[9, 2] loss: 0.277
[9, 7] loss: 0.692
[9, 12] loss: 0.691
Finished epoch #9 val_loss: 0.6891 val_acc: 0.5270
[10, 2] loss: 0.277

In [41]:

def inspect_data(X_np, y_np, name="dataset"):
    print(f"{name} shape: X={X_np.shape}, y={y_np.shape}, dtype X={X_np.dtype}, y={y_np.dtype}")
    # label distribution
    try:
        labels, counts = np.unique(y_np, return_counts=True)
        print("label distribution:", dict(zip(labels.tolist(), counts.tolist())))
    except Exception:
        print("Could not compute label distribution (maybe non-hashable labels).")
    # NaN/Inf checks
    #print("NaNs in X:", np.isnan(X_np).any(), "Infs in X:", np.isinf(X_np).any())
    #print("NaNs in y:", np.isnan(y_np).any() if np.issubdtype(y_np.dtype, np.number) else "non-numeric")
    # feature stats
    means = np.nanmean(X_np, axis=0)
    stds = np.nanstd(X_np, axis=0)
    print("feature means (first 8):", means[:8])
    print("feature stds (first 8):", stds[:8])
    low_var = np.sum(stds < 1e-6)
    print(f"features with near-zero std: {low_var}/{X_np.shape[1]}")

def train_with_debug(model, train_loader, val_loader, n_epochs=20, device=None):
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(1, n_epochs + 1):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0
        grad_norm = 0.0
        batches = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            # quick checks
            if torch.isnan(inputs).any() or torch.isinf(inputs).any():
                raise RuntimeError("NaN/Inf in inputs")
            if not torch.isfinite(inputs).all():
                raise RuntimeError("Non-finite inputs")

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            # gradient norm (for debugging vanishing/exploding grads)
            total_norm = 0.0
            for p in model.parameters():
                if p.grad is not None:
                    param_norm = p.grad.data.norm(2).item()
                    total_norm += param_norm ** 2
            total_norm = total_norm ** 0.5
            grad_norm += total_norm

            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            batches += 1

        scheduler.step()
        avg_grad = grad_norm / batches if batches > 0 else 0.0
        avg_train_loss = train_loss / total if total > 0 else float("nan")
        train_acc = correct / total if total > 0 else 0.0

        # validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                preds = outputs.argmax(dim=1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        avg_val_loss = val_loss / val_total if val_total > 0 else float("nan")
        val_acc = val_correct / val_total if val_total > 0 else 0.0

        print(f"Epoch {epoch:02d}: train_loss={avg_train_loss:.4f} train_acc={train_acc:.4f} "
              f"val_loss={avg_val_loss:.4f} val_acc={val_acc:.4f} grad_norm={avg_grad:.4e} lr={scheduler.get_last_lr()[0]:.1e}")

    return model

# Example usage:
# inspect_data(X_train_np, y_train, name="train")
# inspect_data(X_val_np, y_val, name="val")
# model = GatingNet()  # your model

In [42]:
inspect_data(X_train_np, y_train, name="train")

train shape: X=(1470, 14), y=torch.Size([1470]), dtype X=float64, y=torch.int64
label distribution: {0: 735, 1: 735}
feature means (first 8): [-2.22396263e-01  2.02543814e-01  4.71539207e-01  6.70276517e-04
  7.01982283e-01  1.02405585e-02  3.06549932e-03 -3.02976533e-01]
feature stds (first 8): [0.44791883 0.52822571 0.70334009 0.00106279 1.04807    0.03623345
 0.00138239 0.48572023]
features with near-zero std: 0/14
